<a href="https://colab.research.google.com/github/areebaxijaz/AER850-Project-2/blob/main/AER850Project2Part1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1: Steps 1-4 (Data Processing, Model Building, Training, and Evaluation)**

## Step 1: Data Processing: Use ImageDataGenerator from Keras to augment and load the images.

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Project 2 Data.zip to Project 2 Data (1).zip


In [4]:
!unzip "Project 2 Data.zip"

Archive:  Project 2 Data.zip
replace Data/test/crack/IMG_20230511_101043_jpg.rf.0f754b4a1df6afcfed04bb8468a5f2cb.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
# Importing required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# Set the image dimensions and batch size for training
IMG_WIDTH, IMG_HEIGHT = 500, 500  # We want to resize images to 500x500
BATCH_SIZE = 32  # The batch size is set to 32 for training

# Define the directories where the data is stored
train_dir = '/content/Data/train'  # Path to training data
validation_dir = '/content/Data/valid'  # Path to validation data
test_dir = '/content/Data/test'  # Path to test data

In [7]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

Found 1942 files belonging to 3 classes.
Found 431 files belonging to 3 classes.


In [8]:
# Data augmentation for training data
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),         # Rescaling images
    tf.keras.layers.RandomFlip("horizontal"),  # Randomly flip images horizontally
    tf.keras.layers.RandomRotation(0.1),       # Randomly rotate images
    tf.keras.layers.RandomZoom(0.1)            # Randomly zoom images
])

In [ ]:
# Only rescaling for validation data
validation_dataset = validation_dataset.map(lambda x, y: (x / 255.0, y))

# Apply data augmentation to the training dataset
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

## Step 2: Neural Network Architecture Design

In [9]:
# Define the model
model = Sequential()

# Convolutional layers with MaxPooling and Dropout
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the results and add dense layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # 3 output classes: crack, missing-head, paint-off

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Step 3: Hyperparameter Analysis

In [11]:
# Compile the model with chosen hyperparameters
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Step 4: Model Evaluation

In [12]:
# Train the model and store the training history
epochs = 20
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset
)

Epoch 1/20
 2/61 ━━━━━━━━━━━━━━━━━━━━ 20:19 21s/step - accuracy: 0.2109 - loss: 475.8267

KeyboardInterrupt: 